In [1]:
% matplotlib inline

In [2]:
import numpy as np
import matplotlib.pyplot as plt
import model_list, models, fitting
import matplotlib.lines as mlines
import corner
import copy as cp

from utils import rj2cmb

In [3]:
mean_beta = 1.6
mean_temp = 20.
sigma_beta = .2
sigma_temp = 4.

In [4]:
test = model_list.prob1mbb_model
control = model_list.dust_model

In [5]:
DUST_I = 50.
DUST_P = 5. / 1.41
amp_I=rj2cmb(353e9, DUST_I)
amp_Q=rj2cmb(353e9, DUST_P)
amp_U=rj2cmb(353e9, DUST_P)

test_smallvar = models.ProbSingleMBB(amp_I=rj2cmb(353e9, DUST_I),
                             amp_Q=rj2cmb(353e9, DUST_P),
                             amp_U=rj2cmb(353e9, DUST_P),
                             dust_beta=1.6, dust_T=20.,
                             sigma_beta=.1 * sigma_beta, sigma_temp=.1 * sigma_temp)

In [6]:
nu = np.logspace(np.log10(30), np.log10(500), 7) * 1e9

In [ ]:
amps_I = np.linspace(amp_I + 25, amp_I - 25, 500)
#amps_Q = np.linspace(1, 100, 500)
#amps_U = np.linspace(1, 100, 500)

In [ ]:
def joint_chi2(nu, data_class, model_class, amp_I, amp_Q, amp_U, dust_beta, dust_T, sigma_beta, sigma_temp,
               fsigma_T=1., fsigma_P=10e3):
    
    data_class.amp_I = amp_I
    data_class.amp_Q = amp_Q
    data_class.amp_U = amp_U
    data_class.set_params((dust_beta, dust_T, sigma_beta, sigma_temp))

    beam_mat = np.identity(3*len(nu)) # Beam model
    params = [rj2cmb(353e9, DUST_I), rj2cmb(353e9, DUST_P), rj2cmb(353e9, DUST_P), dust_beta,
              dust_T, sigma_beta, sigma_temp]
    pnames = ('amp_I','dust_beta', 'dust_T', 'sigma_beta', 'sigma_temp')
    initial_vals = (amp_I, dust_beta, dust_T, sigma_beta, sigma_temp)
    parent_model = 'mbb'

    D_vec, Ninv = fitting.generate_data(nu, fsigma_T, fsigma_P, [data_class])

    data_spec = (nu, D_vec, Ninv, beam_mat)
    param_spec = (pnames, initial_vals, parent_model)
    
    likelihood = np.asarray(fitting.lnprob_joint(params, data_spec, [model_class], param_spec))
    
    return likelihood[0]
    
    

In [ ]:
I_test = test.scaling(nu)[0] * test.amps()[0]
I_control = control.scaling(nu)[0] * control.amps()[0]
I_data = np.array(D_vec[:7]).flatten()
err_I = 1./np.sqrt(np.diag(Ninv))[:7]

In [ ]:
#plt.errorbar(nu/1e9, I_test - I_data, yerr=err_I)
plt.errorbar(nu/1e9, np.abs(I_control - I_data), yerr=err_I)


In [ ]:
print plt.rcParams['axes.prop_cycle'].by_key()['color']

In [ ]:
realizations = 1000
chi2s = np.zeros(realizations)

for i in range(realizations):   
    beam_mat = np.identity(3*len(nu)) # Beam model
    params = [amp_I, test.amp_Q, test.amp_U, test.dust_beta,
              test.dust_T, test.sigma_beta, test.sigma_temp]
    pnames = ('dust_beta', 'dust_T', 'sigma_beta', 'sigma_temp')
    initial_vals = (mean_beta, mean_temp, sigma_beta, sigma_temp)
    parent_model = 'mbb'
    D_vec, Ninv = fitting.generate_data(nu, fsigma_T, fsigma_P, [test])
    data_spec = (nu, D_vec, Ninv, beam_mat)
    param_spec = (pnames, initial_vals, parent_model)
    chi2s[i] = fitting.lnprob_joint(params, data_spec, [control], param_spec)
    

In [ ]:
2. * chi2s.sum() / realizations + len(D_vec)

In [ ]:
fig, (ax1, ax2, ax3) = plt.subplots(1,3)
fig.set_figwidth(12)
fig.set_figheight(3)

ax1.hist(2 * chi2s + len(D_vec))
ax1.set_title('fitting MBB to MBB')

ax2.hist(2 * chi2s1 + len(D_vec))
ax2.set_title('fitting high var to MBB')

ax3.hist(2 * chi2s2 + len(D_vec))
ax3.set_title('fitting low var to MBB')

In [12]:
models_fit = [control] 
amp_names = []
param_names = []

for mod in models_fit:
        # Parameter names
        amp_names += ["%s_%s" % (mod.model, pol) for pol in "IQU"]
        param_names += mod.param_names

In [13]:
pnames_MBB = cp.copy(amp_names + param_names)
pnames_MBB

['mbb_I', 'mbb_Q', 'mbb_U', 'dust_beta', 'dust_T']

In [14]:
pnames_MBB, pnames_probMBB

(['mbb_I', 'mbb_Q', 'mbb_U', 'dust_beta', 'dust_T'],
 ['prob1mbb_I',
  'prob1mbb_Q',
  'prob1mbb_U',
  'dust_beta',
  'dust_T',
  'sigma_beta',
  'sigma_temp'])

In [15]:
fsigma_T=1.
fsigma_P=10e3

beam_mat = np.identity(3*len(nu)) # Beam model
params_MBB = [control.amp_I, control.amp_Q, control.amp_U, control.dust_beta,
              control.dust_T]

params_probMBB = [test.amp_I, test.amp_Q, test.amp_U, test.dust_beta,
              test.dust_T, test.sigma_beta, test.sigma_temp]

initial_vals_MBB = (amp_I, amp_Q, amp_U, mean_beta, mean_temp)
initial_vals_probMBB = (amp_I, amp_Q, amp_U,
                        mean_beta, mean_temp, sigma_beta, sigma_temp)
parent_model = 'mbb'

D_vec_MBB, Ninv = fitting.generate_data(nu, fsigma_T, fsigma_P, [control])
D_vec_probMBB, Ninv = fitting.generate_data(nu, fsigma_T, fsigma_P, [test])

data_spec_MBB = (nu, D_vec_MBB, Ninv, beam_mat)
data_spec_probMBB = (nu, D_vec_probMBB, Ninv, beam_mat)

p_spec_MBB = (pnames_MBB, initial_vals_MBB, parent_model)
p_spec_probMBB = (pnames_probMBB, initial_vals_probMBB, parent_model)
    
fitting.lnprob_joint(params_MBB, data_spec_MBB, [control], p_spec_MBB), fitting.lnprob_joint(params_probMBB, data_spec_probMBB, [test], p_spec_probMBB)

(matrix([[-9.00139569]]), matrix([[-9.34052532]]))

In [16]:
pnames_out_control, samples_control, logp_control  = fitting.joint_mcmc(data_spec_MBB, [control], p_spec_MBB, nwalkers=20, 
               burn=100, steps=10000, nthreads=2, sample_file=None)



In [ ]:
pnames_out_test, samples_test, logp_test  = fitting.joint_mcmc(data_spec_probMBB, [test], p_spec_probMBB, nwalkers=20, 
               burn=100, steps=10000, nthreads=2, sample_file=None)



In [ ]:
samples_control = samples_control.reshape(5, 20, 5000)
samples_test = samples_test.reshape(7, 20, 5000)

In [ ]:
np.shape(samples_control.reshape(5, 20*5000))

In [ ]:
fig, ((ax1, ax2, ax3, ax4, ax5), (ax6, ax7, ax8, ax9, ax10)) = plt.subplots(2, 5)
fig.set_figwidth(20)
fig.set_figheight(10)

for i in range(0,20):
    ax1.plot(samples_control[0][i])
    ax2.plot(samples_control[1][i])
    ax3.plot(samples_control[2][i])
    ax4.plot(samples_control[3][i])
    ax5.plot(samples_control[4][i])
    

ax6.hist(samples_control[0].flatten())
ax7.hist(samples_control[1].flatten())
ax8.hist(samples_control[2].flatten())
ax9.hist(samples_control[3].flatten())
ax10.hist(samples_control[4].flatten())
    
ax1.axhline(amp_I)
ax2.axhline(amp_Q)
ax3.axhline(amp_U)
ax4.axhline(mean_beta)
ax5.axhline(mean_temp)

ax6.axvline(amp_I, c='k')
ax7.axvline(amp_Q, c='k')
ax8.axvline(amp_U, c='k')
ax9.axvline(mean_beta, c='k')
ax10.axvline(mean_temp, c='k')

ax1.set_title('amp I')
ax2.set_title('amp Q (noisey)')
ax3.set_title('amp U (noisey)')
ax4.set_title('mean beta')
ax5.set_title('mean temp')

ax6.set_title('amp I')
ax7.set_title('amp Q (noisey)')
ax8.set_title('amp U (noisey)')
ax9.set_title('mean beta')
ax10.set_title('mean temp')

In [ ]:
corner.corner(samples_control.reshape(20 * 5000, 5))

In [ ]:
fig, ((ax1, ax2, ax3, ax4), (ax5, ax6, ax7, ax8)) = plt.subplots(2, 4)
fig.set_figwidth(15)
fig.set_figheight(10)

for i in range(0,20):
    ax1.plot(samples_test[0][i])
    ax2.plot(samples_test[1][i])
    ax3.plot(samples_test[2][i])
    ax4.plot(samples_test[3][i])
    ax5.plot(samples_test[4][i])
    ax6.plot(samples_test[5][i])
    ax7.plot(samples_test[6][i])
    
ax1.axhline(amp_I)
ax2.axhline(amp_Q)
ax3.axhline(amp_U)
ax4.axhline(mean_beta)
ax5.axhline(mean_temp)
ax6.axhline(sigma_beta)
ax7.axhline(sigma_temp)

ax1.set_title('amp I')
ax2.set_title('amp Q (noisey)')
ax3.set_title('amp U (noisey)')
ax4.set_title('mean beta')
ax5.set_title('mean temp')
ax6.set_title('sigma beta')
ax7.set_title('sigma temp')

In [ ]:
fig, ((ax1, ax2, ax3, ax4), (ax5, ax6, ax7, ax8)) = plt.subplots(2, 4)
fig.set_figwidth(15)
fig.set_figheight(10)

ax1.hist(samples_test[0].flatten())
ax2.hist(samples_test[1].flatten())
ax3.hist(samples_test[2].flatten())
ax4.hist(samples_test[3].flatten())
ax5.hist(samples_test[4].flatten())
ax6.hist(samples_test[5].flatten())
ax7.hist(samples_test[6].flatten())
    
ax1.axvline(amp_I, c='k')
ax2.axvline(amp_Q, c='k')
ax3.axvline(amp_U, c='k')
ax4.axvline(mean_beta, c='k')
ax5.axvline(mean_temp, c='k')
ax6.axvline(sigma_beta, c='k')
ax7.axvline(sigma_temp, c='k')

ax1.set_title('amp I')
ax2.set_title('amp Q (noisey)')
ax3.set_title('amp U (noisey)')
ax4.set_title('mean beta')
ax5.set_title('mean temp')
ax6.set_title('sigma beta')
ax7.set_title('sigma temp')

In [ ]:
plt.hist(samples_test[0].flatten())

In [ ]:
logp_control.sum() / len(logp_control), logp_test.sum() / len(logp_test)

In [ ]:
plt.plot(samples[0,:], samples[3, :], 'r.', alpha=0.2)
plt.plot(amp_I, mean_beta, 'kx')
print np.std(samples[0])

In [ ]:
fig, (ax1, ax2) = plt.subplots(2, 1)
fig.set_figwidth(15)
fig.set_figheight(6)

ax1.plot(logp_control)
ax1.set_title('logp: control')

for i in range(20):
    ax1.axvline(i * 5000, c='k')

ax2.plot(logp_test)

for i in range(20):
    ax2.axvline(i * 5000, c='k')
    
ax2.set_title('logp: test')

In [ ]:
np.shape(logp_control)

In [ ]:
fsigma_T=1.
fsigma_P=10e3

beam_mat = np.identity(3*len(nu)) # Beam model
params_MBB = [control.amp_I, control.amp_Q, control.amp_U, control.dust_beta,
              control.dust_T]

params_probMBB = [test.amp_I, test.amp_Q, test.amp_U, test.dust_beta,
              test.dust_T, test.sigma_beta, test.sigma_temp]

initial_vals_MBB = (amp_I, amp_Q, amp_U, mean_beta, mean_temp)
initial_vals_probMBB = (amp_I, amp_Q, amp_U,
                        mean_beta, mean_temp, sigma_beta, sigma_temp)
parent_model = 'mbb'

D_vec_MBB_Planck, Ninv_Planck = fitting.generate_data(nu, fsigma_T, fsigma_P, [control], noise_file='data/noise_planck.dat')
D_vec_probMBB_Planck, Ninv_Planck = fitting.generate_data(nu, fsigma_T, fsigma_P, [test], noise_file='data/noise_planck.dat')

data_spec_MBB_Planck = (nu, D_vec_MBB, Ninv_Planck, beam_mat)
data_spec_probMBB_Planck = (nu, D_vec_probMBB, Ninv_Planck, beam_mat)

p_spec_MBB = (pnames_MBB, initial_vals_MBB, parent_model)
p_spec_probMBB = (pnames_probMBB, initial_vals_probMBB, parent_model)
    
fitting.lnprob_joint(params_MBB, data_spec_MBB_Planck, [control], p_spec_MBB), fitting.lnprob_joint(params_probMBB, data_spec_probMBB_Planck, [test], p_spec_probMBB)


In [ ]:
import time
t0 = time.time()
fitting.lnprob_joint(params_probMBB, data_spec_probMBB_Planck, [test], p_spec_probMBB)
print time.time() - t0


In [ ]:
fitting.lnprob_joint(params_MBB, data_spec_MBB_Planck, [control], p_spec_MBB), fitting.lnprob_joint(params_MBB, data_spec_MBB, [control], p_spec_MBB)

In [ ]:
fitting.lnprob_joint(params_MBB, data_spec_probMBB_Planck, [control], p_spec_MBB), fitting.lnprob_joint(params_MBB, data_spec_probMBB, [control], p_spec_MBB)

In [ ]:
plt.figure(figsize=(10,7))
plt.errorbar(nu/1e9, test.scaling(nu)[0],
             yerr=1./np.sqrt(np.diag(Ninv))[:7], lw=3)
plt.errorbar(nu/1e9, test.scaling(nu)[0],
             yerr=1./np.sqrt(np.diag(Ninv_Planck))[:7], lw=3, alpha=.5)

In [ ]:
I_test = test.scaling(nu)[0] * test.amps()[0]
I_control = control.scaling(nu)[0] * control.amps()[0]
I_data = np.array(D_vec[:7]).flatten()
err_I = 1./np.sqrt(np.diag(Ninv))[:7]

In [ ]:
x, y = np.random.multivariate_normal((0., 1.), np.eye(2), size=5000).T

In [ ]:
plt.plot(x,y, 'r.')

In [ ]:
ax = corner.corner(np.column_stack((x,y)))

In [ ]:
s = np.column_stack((samples_control[3].flatten(), samples_control[4].flatten()))

In [ ]:
samples_control.shape

In [ ]:
s_T = samples_test.reshape((7, 100000)).T
s_T.shape

In [ ]:
ax = corner.corner(s_T, plot_datapoints=False)

In [ ]:
mean_beta, mean_temp

In [ ]:
sigma_beta, sigma_temp